## Libraries

In [2]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory

## Initial LLM

In [3]:
llm = ChatOllama(
    base_url="http://localhost:11434",
    model="llama3.1:8b",
    temperature=.7
)

prompt_temp = ChatPromptTemplate.from_messages([
    ("placeholder", "{history}"),
    ("system", "تو یک دستیار هوشمند هستی"),
    ("human", "{prompt}")
])

def get_history(session_id="Wreza"):
    return SQLChatMessageHistory(session_id=session_id, connection="sqlite:///rag.db")


chain = prompt_temp | llm | StrOutputParser()

history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_history,
    input_messages_key="prompt",
    history_messages_key="history"
)

## Load and Split PDF file

In [1]:
!pip install pypdf --quiet

In [13]:
from langchain_community.document_loaders import PyPDFLoader

pdfs = ["pdf/1.pdf", "pdf/2.pdf", "pdf/3.pdf"]
documents = []

for pdf in pdfs:
    documents.extend(PyPDFLoader(pdf).load())

len(documents)

99

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)

all_splits = text_splitter.split_documents(documents)

len(all_splits)

428

## Embedding

In [15]:
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(model="llama3.1:8b")

vect1 = embedding.embed_query(all_splits[0].page_content)
vect2 = embedding.embed_query(all_splits[0].page_content)

assert len(vect1) == len(vect2)
print(len(vect1))

4096


## Vector Stores

In [7]:
!pip install -qU langchain-chroma

In [16]:
from langchain_chroma import Chroma

vector_store = Chroma.from_documents(
    documents=all_splits,
    embedding=embedding,
    persist_directory="./vector_db"
)

## Retriving from the Persistant Vector Database

In [18]:
vector_store = Chroma(
    persist_directory="./vector_db",
    embedding_function=embedding
    )

result = vector_store.similarity_search("federated learning", k=3)

for doc in result:
    print(doc.page_content)
    print("-"*30)

via functional encryption, revisited. IEEE Transactions on Infor mation Forensics
and Security 18, 1855–1869 (2023)
[179] Hijazi, N.M., Aloqaily, M., Guizani, M., Ouni, B., Karray, F.: Secure fe der-
ated learning with fully homomorphic encryption for iot communication s. IEEE
Internet of Things Journal (2023)
[180] Zhao, P., Cao, Z., Jiang, J., Gao, F.: Practical private aggrega tion in federated
41
------------------------------
current knowledge to maximize performance. Then, they propose d FedHQL, where the
local agents update their action-value independently based on Q-le arning. The central
server plays a crucial role in coordinating the exchange of knowledg e between agents by
broadcasting, receiving action-value estimates, and selecting act ions with the highest
UCB (Upper Conﬁdence Bound) [ 138] value. There are still many ongoing explorations
in other areas where FedRL is being applied, including energy managem ent [ 139, 140],
electric vehicle charging and uncharging [ 141].

In [9]:
# session_id = "Wreza"

# response = history.invoke({"prompt": "what is an apple"}, config={"configurable":{"session_id": session_id}})

# print(response)